In [ ]:
%load_ext autoreload
%autoreload 2

import functools
import numpy as np

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow as tf
import tensorflow_privacy as tf_privacy

import tsgm

import matplotlib.pyplot as plt

# Notes

## Idea
- standard GANs for time series generation with recurrent architectures,
- decoder loss average cross-entropy between sequences:
$$\mathrm{D}_{\text {loss }}\left(X_{n}, \mathbf{y}_{n}\right)=-\mathrm{CE}\left(\mathrm{RNN}_{\mathrm{D}}\left(X_{n}\right), \mathbf{y}_{n}\right)$$

- generator tries to trick the descriminator, and its loss:
$$\mathrm{G}_{\mathrm{loss}}\left(Z_{n}\right)=\mathrm{D}_{\mathrm{loss}}\left(\mathrm{RNN}_{\mathrm{G}}\left(Z_{n}\right), \mathbf{1}\right)=-\mathrm{CE}\left(\mathrm{RNN}_{\mathrm{D}}\left(\mathrm{RNN}_{\mathrm{G}}\left(Z_{n}\right)\right), \mathbf{1}\right)$$

- in conditional GANs, class labels are concatenated to latent vectors:
$$\mathbf{Z}_{n t} \rightarrow\left[\mathbf{Z}_{n t} ; \mathbf{c}_{n}\right] \quad \mathbf{x}_{n t} \rightarrow\left[\mathbf{x}_{n t} ; \mathbf{c}_{n}\right]$$


## Evaluation
- MMD ($V$ is the asymptotic variance of $\text{MMD}^2$):
$${\widehat{\mathrm{MMD}_{u}}}^{2}=\frac{1}{n(n-1)} \sum_{i=1}^{n} \sum_{j \neq i}^{n} K\left(x_{i}, x_{j}\right)-\frac{2}{m n} \sum_{i=1}^{n} \sum_{j=1}^{m} K\left(x_{i}, y_{j}\right)+\frac{1}{m(m-1)} \sum_{i=1}^{m} \sum_{j \neq i}^{m} K\left(y_{i}, y_{j}\right)$$
$$\hat{t}=\frac{\widehat{\mathrm{MMD}}^{2}}{\sqrt{\hat{V}}}$$
- train on synthetic, test on real (TSTR)
- train on real, test on synthetic (TRTS)

## Experiments
- Sine Waves
- Smooth functions (Samples from GPs)
- MNIST as a time-series
- ICU data

In [ ]:
latent_dim = 64
output_dim = 2
feature_dim = 1
seq_len = 100
batch_size = 128


generator_in_channels = latent_dim + output_dim
discriminator_in_channels = feature_dim + output_dim

In [ ]:
X, y_i = tsgm.utils.gen_sine_vs_const_dataset(5_000, seq_len, 1, max_value=20, const=10)

scaler = tsgm.utils.TSFeatureWiseScaler((-1, 1))
X_train = scaler.fit_transform(X)
y = keras.utils.to_categorical(y_i, 2)

X_train = X_train.astype(np.float32)
y = y.astype(np.float32)

dataset = tf.data.Dataset.from_tensor_slices((X_train, y))
dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)

In [ ]:
tsgm.utils.visualize_ts_lineplot(X_train, y_i, num=2)

In [ ]:
architecture = tsgm.models.architectures.cGAN_LSTMnArchitecture(
    seq_len=seq_len, feat_dim=feature_dim,
    latent_dim=latent_dim, output_dim=output_dim)
discriminator, generator = architecture._discriminator, architecture._generator

In [ ]:
architecture._discriminator.summary()

In [ ]:
architecture._generator.summary()

In [ ]:
cond_gan = tsgm.models.cgan.ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0002, beta_1=0.5),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

cbk = tsgm.models.monitors.GANMonitor(num_samples=3, latent_dim=latent_dim, save=False, save_path="/tmp", labels=y)
cond_gan.fit(dataset, epochs=2, callbacks=[cbk])

In [ ]:
X_gen = cond_gan.generate(y)

In [ ]:
statistics = [functools.partial(tsgm.metrics.statistics.axis_max_s, axis=1),
              functools.partial(tsgm.metrics.statistics.axis_min_s, axis=1)]

sim_metric = tsgm.metrics.DistanceMetric(
    statistics=statistics, discrepancy=lambda x, y: np.linalg.norm(x - y)
)

print(f"Distance metric: {sim_metric(X, X_gen)}")

## Privacy preserving RCGAN

In [ ]:
l2_norm_clip = 1.5
noise_multiplier = 1.3
num_microbatches = 1
learning_rate = 0.01


d_optimizer = tf_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
)


g_optimizer = tf_privacy.DPKerasSGDOptimizer(
    l2_norm_clip=l2_norm_clip,
    noise_multiplier=noise_multiplier,
    num_microbatches=num_microbatches,
    learning_rate=learning_rate
)

In [ ]:
cond_gan = tsgm.models.cgan.ConditionalGAN(
    discriminator=discriminator, generator=generator, latent_dim=latent_dim
)
cond_gan.compile(
    d_optimizer=d_optimizer,
    g_optimizer=g_optimizer,
    loss_fn=keras.losses.BinaryCrossentropy(),
)

cbk = tsgm.models.monitors.GANMonitor(num_samples=3, latent_dim=latent_dim, save=False, save_path="/tmp", labels=y)
cond_gan.fit(dataset, epochs=2, callbacks=[cbk])